In [1]:
import sys, os

# このノートブックが test/ にあると仮定
notebook_dir = os.getcwd()
project_root = os.path.abspath(os.path.join(notebook_dir, ".."))

# GraphicalPR をモジュール検索パスに追加
if project_root not in sys.path:
    sys.path.insert(0, project_root)

# パス追加後にインポート
from graph import Wave
from core import UncertainArray

# テスト実行
w = Wave()
print(w.belief)

None


In [1]:
import sys, os
sys.path.insert(0, os.path.abspath(".."))

import numpy as np
from core.uncertain_array import UncertainArray
from graph.wave import Wave


# Dummy factor class for testing
class DummyFactor:
    def __init__(self, name):
        self.name = name
        self.last_received_message = None

    def receive_message(self, wave, message):
        print(f"{self.name} received message from Wave: shape={message.shape}")
        self.last_received_message = message

# Create a Wave with shape (2, 2)
wave = Wave(shape=(2, 2), dtype=np.complex128)

# Create dummy parent and children
parent = DummyFactor("ParentFactor")
child1 = DummyFactor("ChildFactor1")
child2 = DummyFactor("ChildFactor2")

# Create UncertainArray messages
msg_from_parent = UncertainArray.random((2, 2), precision=2.0)
msg_from_child1 = UncertainArray.random((2, 2), precision=1.0)
msg_from_child2 = UncertainArray.random((2, 2), precision=1.5)

# Connect parent and children to the wave
wave.add_parent(parent, msg_from_parent)
wave.add_child(child1, msg_from_child1)
wave.add_child(child2, msg_from_child2)

# Compute belief explicitly and inspect
belief = wave.compute_belief()
print("Belief computed:")
print(belief.data)
print("Belief precision:")
print(belief.precision)

# Run forward pass and check child factors received messages
wave.forward()
assert child1.last_received_message is not None
assert child2.last_received_message is not None

# Run backward pass and check parent received message
wave.backward()
assert parent.last_received_message is not None
print("Backward message received by parent factor.")


Belief computed:
[[ 0.4252217 -0.66334796j  0.26345419+0.42751126j]
 [-0.58450122+1.31164114j  0.40269413-0.01188743j]]
Belief precision:
[[4.5 4.5]
 [4.5 4.5]]
ChildFactor1 received message from Wave: shape=(2, 2)
ChildFactor2 received message from Wave: shape=(2, 2)
ParentFactor received message from Wave: shape=(2, 2)
Backward message received by parent factor.
